In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.datasets import mnist

2022-08-05 18:36:20.787489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-05 18:36:20.787546: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
